# Cvičenie 7: Wordle pomocou OOP

Na dnešnom cvičení sa vrátime k našej implementácii hry Wordle z 5. týždňa. Ak vlastné riešenie nemáte po ruke, [môžete si stiahnuť ukážkové riešenie](sources/lab07/wordle_solution.py). Samozrejme k hre naďalej potrebujeme aj [zoznam akceptovaných slov, ktorý nájdete v tomto súbore](sources/lab07/word_list.txt). Stiahnite si teda tieto súbory a vyskúšajte si, či všetko funguje. Ak si už nepamätáte, ako sa hra Wordle riešila, naštudujte si rýchlo kód.

V prvej časti cvičenia sa budeme snažiť naše riešenie upraviť tak, aby využívala objektovo orientované programovanie, teda namiesto funkcií náš kód rozdelíme do samostatných modulov, teda **tried**, pričom každá trieda bude modelovať jeden dôležitý prvok celkového riešenia. Rýchlo si zopakujte teoretické poznatky ohľadom tried a objektov, a následne navrhnite niekoľko tried, ktoré by sme mohli využívať na reprezentáciu rôznych komponentov hry.

Pri návrhu vám môže pomôcť zoznam funkcií, ktoré sme implementovali pred dvomi týždňami. Skúste sa porozmýšľať o tom, aké rozdelenie funkcionality by dávalo zmysel pre objektové riešenie:

* `load_words(dataset_path)` - načíta zoznam povolených slov zo súboru, cestu ku ktorému dostane ako parameter;
* `get_puzzle(word_list)` - zo zoznamu slov `word_list` vráti náhodný prvok, ktorý poslúži ako hľadané slovo;
* `is_game_finished(guess, puzzle)` - skontroluje, či používateľ už uhádol hľadané slovo;
* `evaluate_guess(guess, puzzle)` - vyhodnotí tip používateľa a vráti informáciu o jednotlivých písmenách v slove;
* `start_game(dataset_path)` - nasimuluje jednu hru s ľudským hráčom, ktorý zadáva svoje tipy cez konzolu (štandardný vstup).

## 1. Objektové riešenie

Na konci návrhu možno ste identifikovali dva hlavné komponenty, a to hľadané slovo (`Puzzle`) a samotnú hru (`Game`). Keďže od hráča potrebujeme iba vstup, zatiaľ pre reprezentáciu hráča nepotrebujeme špeciálnu triedu.

Skúste pretransformovať vaše procedurálne riešenie do objektovo orientovaného programovania, pričom rozdelenie funkcionality je nasledovné:

* `Puzzle`:

    * `is_game_finished(self, guess)`
    * `evaluate_guess(self, guess)`

* `Game`:
    * `load_words(self, dataset_path)`
    * `generate_puzzle(self)` - podobne ako `get_puzzle()`, avšak bez návratovej hodnoty, iba vytvorí objekt typu `Puzzle`
    * `start_game(self)`

[Kostru riešenia nájdete aj ako python skript.](sources/lab07/wordle_oop.py)

**Poznámka:** Ak ste navrhli iné riešenie, pokojne môžete funkcionalitu implementovať iným štýlom, na konci môžete porovnať rôzne riešenia.

In [ ]:
import random


class Puzzle:
    def __init__(self, word):
        self.puzzle = word.lower()

    def is_game_finished(self, guess):
        return False

    def evaluate_guess(self, guess):
        return list()


class Game:
    def __init__(self, dataset_path):
        self.word_list = self.load_words(dataset_path)
        self.puzzle = None

    def load_words(self, dataset_path):
        return list()

    def generate_puzzle(self):
        pass

    def start_game(self):
        pass

Po doimplementácii funkcionality môžete vaše riešenie otestovať pomocou ďalšieho bloku kódu:

In [ ]:
wordle = Game("word_list.txt")
wordle.start_game()

## 2. Boti na scéne

Je známym faktom, že programátori sú leniví, a aj keď je Wordle zábavná hra a skvelý spôsob na prokrastináciu, skôr či neskôr im napadne implementovať inteligentného bota, ktorý túto hru zahrá za nich omnoho efektívnejšie. V tejto úlohe vytvoríme jedného takéhoto bota - samozrejme cez definíciu ďalšej triedy.

In [ ]:
from copy import deepcopy
import string


PLAYER_KNOWLEDGE = [(letter, None, -1)
                    for letter in string.ascii_lowercase]


class Bot:
    def __init__(self, word_list):
        self.reset(word_list)

    def reset(self, word_list):
        pass

    def get_player_guess(self):
        return ""

    def process_result(self, result):
        pass

### 2.1. Bot sa pripravuje

Ako môžete vidieť, na rozdiel od predošlých tried, v konštruktore `Bot` sa nenastaví žiadna hodnota priamo, namiesto toho sa zavolá iba metóda `reset()`, ktorá má pripraviť bota na riešenie úlohy. Náš bot bude daný dvomi hodnotami, ktoré musíte inicializovať práve v tejto metóde:

* `self.knowledge` - reprezentácia toho, čo bot vie o hľadanom slove (aktualizuje sa na základe spätnej väzby od hry). Bude to zoznam n-tíc vo forme `PLAYER_KNOWLEDGE` (štruktúra popísaná nižšie).
* `self.possibles` - zoznam slov, ktoré sú akceptované hrou (ako výstup `load_words()`).

Znalosti bota budú reprezentované v zozname `knowledge`, ktorý sa má inicializovať ako kópia konštanty `PLAYER_KNOWLEDGE` (hlboká kópia, keďže je to zoznam objektov/n-tíc). Tento zoznam je inicializovaný na začiatku hry a je to zoznam trojíc, kde každá trojica reprezentuje informáciu o jednotlivých možných písmenách s nasledovnou štruktúrou:

 - písmeno – string s dĺžkou 1
 - informácia o tom, či sa písmeno nachádza v slove – inicializovaná na `None`, neskôr nahradíte hodnotu booleovskými hodnotami `True/False`
 - informácia o pozícii písmena v správnom riešení – inicializovaná na -1, neskôr nahradíte platným indexom 0 až 4.
 
**Poznámka:** tento bot nebude rátať s možnosťou viacnásobného výskytu písmena v slove, teda tretia hodnota bude vždy iba jedno číslo.

**Úloha:** Implementujte metódu `reset()` tak, aby inicializovala členské premenné triedy ako kópie konštanty resp. parametra.

### 2.2. Bot háda

Ďalším krokom je implementovať metódu `get_player_guess()`, ktorá najprv zo zoznamu slov `self.possibles` vyberie zoznam možných riešení a vracia náhodný tip. Bot eliminuje možnosti na základe získaných vedomostí o správnom riešení, ktoré sú dostupné v `self.knowledge`.

Na základe týchto vedomostí by mala metóda `get_player_guess` eliminovať možnosti nasledovne:
 1. vymazať zo zoznamu možných riešení slová, ktoré neobsahujú písmená, o ktorých vie, že sa nachádzajú v správnom riešení na základe zoznamu `self.knowledge`
 2. vymazať zo zoznamu možných riešení slová, ktoré obsahujú písmená, o ktorých vie, že sa nenachádzajú v správnom riešení na základe zoznamu `self.knowledge`
 3. vymazať zo zoznamu možných riešení slová, ktoré nemajú správne písmeno na niektorej pozícii, o ktorej už vie, aké tam bude písmeno.

Pre jednoduchosť uvedieme aj jeden príklad. Predpokladajme, že bot vie, že riešenie určite obsahuje písmená *r*, *s*, *e*, *u*; určite neobsahuje písmeno *a*; a na prvej pozícii je písmeno *r*. Na základe jednotlivých pravidiel by teda eliminoval napríklad:
 1. slovo *table*, ktoré neobsahuje potrebné písmená *r*, *s*, *u*
 2. slovo *braid*, ktoré obsahuje písmeno *a*, o ktorom už hráč vie, že sa v riešení nenachádza
 3. slovo *trout*, keďže hráč už vie, že na prvej pozícii musí byť písmeno *r*

Po eliminácii niektorých možných riešení, bot vyberie náhodné slovo zo zoznamu. Pred vrátením náhodného tipu nezabudnite vhodne aktualizovať členskú premennú `self.possibles`.

**Úloha:** Implementujte metódu `get_player_guess()` podľa vyššie uvedenej špecifikácie.

### 2.3. Bot sa učí (?)

Samozrejme bot má šancu hru vyhrať iba vtedy, ak bude postupne aktualizovať svoje vedomosti o hľadanom slove. K tomu slúži metóda `process_result()` s parametrom `result`, ktorý obsahuje spätnú väzbu od hry, ktorá je vygenerovaná metódou `evaluate_guess()` (formát sa nemení). Metóda postupne spracuje informácie o všetkých písmenách z posledného tipu a to nasledovne:
 1. aktualizuje informáciu o tom, či sa písmeno nachádza alebo nenachádza v správnom riešení
 2. ak je písmeno aj na správnej pozícii, aktualizuje tento údaj v zozname `self.knowledge`.

Metóda `process_result()` nemá návratovú hodnotu, aktualizujte priamo zoznam `self.knowledge`.

**Úloha:** Implementujte metódu `process_result(result)` podľa popisu vyššie, a takto dokončite implementáciu vášho bota.

**Poznámka:** Nezabudnite, že zoznam `self.knowledge` obsahuje n-tice, ktoré sú nemenné. Práve preto pri spracovaní spätnej väzby musíte vytvoriť novú n-ticu a uložiť ju na správne miesto v zozname knowledge. Poradie informácií o písmenách v zozname má pritom ostať rovnaké, t.j. najprv budete mať vedomosti o výskyte písmena *a*, písmena *b*, atď.

## 3. Doplnenie hry

Po implementácii triedy `Bot` máme bota hotového, no jeho funkčnosť zatiaľ vyskúšať nevieme, keďže naša hra interakciu s botom nepodporuje.

**Úloha:** Pridajte do triedy `Game` metódu `bot_game(bot)`, ktorá ako parameter dostane objekt bota, ktorý si hru chce zahrať. Metóda je veľmi podobná `start_game()` – vstup od hráča je ale nahradený vstupom od bota. Počas hry môžete takisto vypísať zoznam možných riešení podľa bota, ktorý bude stále kratší ako bot spracuje informácie, ktoré sa dozvie o hľadanom slove.

Vaše riešenie následne môžete otestovať pomocou kódu:

In [ ]:
wordle = Game("word_list.txt")
agent = Bot(wordle.word_list)
wordle.bot_game(agent)